In [1]:
import os
import torch
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.general import GiveUniqueNodeNames, GiveReadableTensorNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.fold_constants import FoldConstants
from finn.util.visualization import showInNetron
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN

# 1. 定义文件名
onnx_file = "lenet_w2a1_mixed.onnx"

build_dir = "output_lenet" # 定义输出目录
if not os.path.exists(build_dir):
    os.makedirs(build_dir)

# 1. 加载模型
model = ModelWrapper(onnx_file)

In [2]:
showInNetron(onnx_file)

Serving 'lenet_w2a1_mixed.onnx' at http://0.0.0.0:8081


In [2]:
# 2. 转换为 FINN 格式 (MultiThreshold)
print("正在转换为 FINN 格式...")
model = model.transform(ConvertQONNXtoFINN())

# 3. 基础清理与常量折叠 (Tidy-up)
# 反复运行以确保所有权重被冻结为 Initializer
print("正在执行清理...")
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

# 保存并查看
model.save(build_dir + "/step1_tidy.onnx")
print("✅ 第一步完成：模型已转换为 FINN 格式并清理完毕。")

正在转换为 FINN 格式...
正在执行清理...
✅ 第一步完成：模型已转换为 FINN 格式并清理完毕。


In [8]:
showInNetron(build_dir + "/step1_tidy.onnx")

Serving 'output_lenet/step1_tidy.onnx' at http://0.0.0.0:8081


In [10]:
from finn.util.pytorch import ToTensor
from finn.util.pytorch import Normalize
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.core.datatype import DataType
from qonnx.transformation.insert_topk import InsertTopK

# 1. 创建预处理模型 (除以 255)
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
totensor_pyt = Normalize(torch.tensor(33.3285,), torch.tensor(78.5655,),1) 
export_path = build_dir + "/preproc.onnx"
# 临时导出一个小模型
from brevitas.export import export_qonnx
export_qonnx(totensor_pyt, torch.randn(ishape), export_path)

# 2. 合并预处理
pre_model = ModelWrapper(export_path)
model = model.transform(MergeONNXModels(pre_model))

# 3. 显式标记输入为 UINT8 (这对 FPGA 很重要)
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])

# 4. 添加后处理 (Top-1)
model = model.transform(InsertTopK(k=1))

# 5. 再次清理
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

model.save(build_dir + "/step2_pre_post.onnx")
print("✅ 第二步完成：已添加预处理和 TopK。")

✅ 第二步完成：已添加预处理和 TopK。


/home/maxwellqin/projects/finn/deps/qonnx/src/qonnx/transformation/merge_onnx_models.py:70: UserWarning: [MergeONNXModels] opsets for models to merge differ: 14 vs 11, output model will use opset 14
  warnings.warn(
/home/maxwellqin/projects/finn/deps/qonnx/src/qonnx/transformation/infer_data_layouts.py:127: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


In [11]:
showInNetron(build_dir + "/step2_pre_post.onnx")

Serving 'output_lenet/step2_pre_post.onnx' at http://0.0.0.0:8081


In [3]:
from finn.transformation.streamline import Streamline
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from qonnx.transformation.bipolar_to_xnor import ConvertBipolarMatMulToXnorPopcount
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC, MoveScalarLinearPastInvariants
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.general import RemoveUnusedTensors

model = ModelWrapper(build_dir + "/step2_pre_post.onnx")
model = model.transform(MoveScalarLinearPastInvariants())
model = model.transform(Streamline())
model = model.transform(LowerConvsToMatMul())
model = model.transform(MakeMaxPoolNHWC())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(Streamline())
# absorb final add-mul nodes into TopK
model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())

model = model.transform(RemoveUnusedTensors())
model.save(build_dir + "/lenet_streamlined.onnx")

# 保存结果
model.save(build_dir + "/lenet_streamlined.onnx")
print("✅ 第三步完成：模型已流线化并转换为 MatMul。")

✅ 第三步完成：模型已流线化并转换为 MatMul。


In [5]:
showInNetron(build_dir + "/lenet_streamlined.onnx")

Serving 'output_lenet/lenet_streamlined.onnx' at http://0.0.0.0:8081


In [12]:
import os
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
from finn.transformation.fpgadataflow.create_dataflow_partition import (
    CreateDataflowPartition,
)
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers
from finn.transformation.streamline.absorb import AbsorbConsecutiveTransposes
from qonnx.custom_op.registry import getCustomOp
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.general import RemoveUnusedTensors, GiveUniqueNodeNames
from qonnx.core.modelwrapper import ModelWrapper
from finn.util.basic import pynq_part_map
# 板卡设置
pynq_board = "Pynq-Z1" 
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10
# ===========================================

model = ModelWrapper(build_dir + "/lenet_streamlined.onnx")

# 1. 将通用层转换为 HW 硬件层
# 处理纯二值化层 (W1A1)
model = model.transform(to_hw.InferBinaryMatrixVectorActivation())
# 处理多比特/混合精度层 (关键: 处理你的 INT2 权重)
model = model.transform(to_hw.InferQuantizedMatrixVectorActivation())

# 2. 处理特定功能层
model = model.transform(to_hw.InferLabelSelectLayer())    # TopK -> LabelSelect
model = model.transform(to_hw.InferThresholdingLayer())   # 阈值层 -> Thresholding_Batch
model = model.transform(to_hw.InferConvInpGen())          # Im2Col -> SlidingWindow
model = model.transform(to_hw.InferStreamingMaxPool())    # MaxPool -> StreamingMaxPool

# 3. 清理连接层 (解决之前的 Flatten/Reshape 问题)
# 这一步会识别 Transpose->Reshape 模式，将其移除并重排后续 MatMul 的权重
print("Removing CNV to FC Flatten/Reshape...")
model = model.transform(RemoveCNVtoFCFlatten())

# 合并连续的 Transpose
model = model.transform(AbsorbConsecutiveTransposes())

# 4. 整理图结构与数据布局
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())
model = model.transform(GiveUniqueNodeNames())

# 5. 数据流分区 (Dataflow Partitioning)
print("Creating Dataflow Partition...")
parent_model = model.transform(CreateDataflowPartition())
parent_model.save(build_dir + "/lenet_dataflow_parent.onnx")

# 6. 提取子图并专门化为 HLS 变体 (Specialize Layers)
# 获取分区后的子图模型文件名
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
dataflow_model_filename = sdp_node.get_nodeattr("model")

# 加载子图并转换为 HLS 层
print(f"Specializing layers for {pynq_board} ({fpga_part})...")
dataflow_model = ModelWrapper(dataflow_model_filename)
dataflow_model = dataflow_model.transform(SpecializeLayers(fpga_part))

# 保存最终结果
save_path = build_dir + "/lenet_dataflow_model.onnx" 
dataflow_model.save(save_path)

print(f"✅ HW conversion complete. Saved to {save_path}")

Removing CNV to FC Flatten/Reshape...
Creating Dataflow Partition...
Specializing layers for Pynq-Z1 (xc7z020clg400-1)...
✅ HW conversion complete. Saved to output_lenet/lenet_dataflow_model.onnx


In [14]:
showInNetron(build_dir + "/lenet_dataflow_model.onnx")

Serving 'output_lenet/lenet_dataflow_model.onnx' at http://0.0.0.0:8081


In [16]:
# 查看父模型（外壳）
showInNetron(build_dir + "/lenet_dataflow_parent.onnx")

Serving 'output_lenet/lenet_dataflow_parent.onnx' at http://0.0.0.0:8081


In [50]:
# 【关键修正】从 qonnx 导入 ModelWrapper，而不是 finn.core
from qonnx.core.modelwrapper import ModelWrapper 
from qonnx.custom_op.registry import getCustomOp
import os

# 1. 加载 Step 4 生成的子模型
# 确保 build_dir 变量存在 (如果在新开的 notebook 里，记得重新定义一下 build_dir)
if 'build_dir' not in locals():
    build_dir = os.environ["FINN_BUILD_DIR"] + "/lenet_build"

model_path = build_dir + "/lenet_dataflow_model.onnx"
model = ModelWrapper(model_path)

print("🚀 开始 Step 5: 配置并行度 (Folding)...")

# ==============================================================================
# A. 配置 MVAU (矩阵向量计算单元)
# ==============================================================================
# 获取所有 HLS 版本的 MVAU
# mvau_layers = model.get_nodes_by_op_type("MVAU_hls")

# for node in mvau_layers:
#     inst = getCustomOp(node)
    
#     # 获取形状: MW=输入宽, MH=输出宽
#     mw = inst.get_nodeattr("MW") 
#     mh = inst.get_nodeattr("MH")
    
#     # --- 自动计算策略 ---
#     # 规则：PE 必须整除 MH，SIMD 必须整除 MW
    
#     # 1. 设置 PE (输出并行度)
#     if mh % 8 == 0: pe = 8
#     elif mh % 4 == 0: pe = 4
#     elif mh % 2 == 0: pe = 2
#     else: pe = 1
        
#     # 2. 设置 SIMD (输入并行度)
#     if mw % 8 == 0: simd = 8
#     elif mw % 4 == 0: simd = 4
#     elif mw % 2 == 0: simd = 2
#     else: simd = 1
        
#     inst.set_nodeattr("PE", pe)
#     inst.set_nodeattr("SIMD", simd)
    
#     # 设置 FIFO
#     inst.set_nodeattr("inFIFODepths", [32])
#     inst.set_nodeattr("outFIFODepths", [32])
    
#     print(f"Layer {node.name}: Shape({mw},{mh}) -> PE={pe}, SIMD={simd}")

from qonnx.custom_op.registry import getCustomOp

fc_layers = model.get_nodes_by_op_type("MVAU_hls")

# 格式: (PE, SIMD, FIFO_depth)
# 注意：我在 FC1 和 Conv2 做了妥协，降低了 SIMD 以确保板卡能跑通
folding = [
    (8, 5, [32]),
    (16, 10, [32]), 
    (8, 8, [32]), 
    (1, 1, [32]), 
    (1, 1, [32]), 
]

for fcl, (pe, simd, fifo_depth) in zip(fc_layers, folding):
    fcl_inst = getCustomOp(fcl)
    
    # 1. 设置并行度
    fcl_inst.set_nodeattr("PE", pe)
    fcl_inst.set_nodeattr("SIMD", simd)
    fcl_inst.set_nodeattr("inFIFODepths", fifo_depth)
    fcl_inst.set_nodeattr("outFIFODepths", fifo_depth) 


# ==============================================================================
# B. 配置 SWG (卷积输入生成器)
# ==============================================================================
# 尝试查找 RTL 或 HLS 版本的节点
swg_layers = model.get_nodes_by_op_type("ConvolutionInputGenerator_rtl") 
if len(swg_layers) == 0:
    swg_layers = model.get_nodes_by_op_type("ConvolutionInputGenerator_hls")

for node in swg_layers:
    inst = getCustomOp(node)
    
    # 安全起见设为 1，确保通过综合
    inst.set_nodeattr("SIMD", 1) 
    inst.set_nodeattr("inFIFODepths", [32])
    inst.set_nodeattr("outFIFODepths", [32])
    print(f"Layer {node.name}: SIMD=1 (Safe Mode)")

# ==============================================================================
# C. 配置 Thresholding (阈值层)
# ==============================================================================
thresh_layers = model.get_nodes_by_op_type("Thresholding_rtl")
if len(thresh_layers) == 0:
     thresh_layers = model.get_nodes_by_op_type("Thresholding_hls")

for node in thresh_layers:
    inst = getCustomOp(node)
    inst.set_nodeattr("PE", 1) 
    inst.set_nodeattr("outFIFODepths", [32])

# ==============================================================================
# 保存结果
# ==============================================================================
model.save(build_dir + "/lenet_folded.onnx")
print(f"✅ Step 5 完成！模型已保存至: {build_dir}/lenet_folded.onnx")

🚀 开始 Step 5: 配置并行度 (Folding)...
Layer : SIMD=1 (Safe Mode)
Layer : SIMD=1 (Safe Mode)
Layer : SIMD=1 (Safe Mode)
✅ Step 5 完成！模型已保存至: output_lenet/lenet_folded.onnx


In [51]:
showInNetron(build_dir + "/lenet_folded.onnx")

Serving 'output_lenet/lenet_folded.onnx' at http://0.0.0.0:8081


In [52]:
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
model = ModelWrapper(build_dir + "/lenet_folded.onnx")
model = model.transform(ZynqBuild(platform = pynq_board, period_ns = target_clk_ns))

/home/maxwellqin/projects/finn/src/finn/transformation/fpgadataflow/floorplan.py:107: UserWarning: 20 nodes have no entry in the provided floorplan, SLR was set to -1
  warnings.warn(
/home/maxwellqin/projects/finn/src/finn/transformation/fpgadataflow/insert_fifo.py:234: UserWarning: Input FIFO for IODMA_hls_0_out0 has depth 2 and won't
                        be created. This may cause RTL simulation issues.
                        
  warnings.warn(
/home/maxwellqin/projects/finn/src/finn/transformation/fpgadataflow/insert_fifo.py:294: UserWarning: Output FIFO for LabelSelect_hls_0_out0 has depth 2 and won't
                        be created. This may cause RTL simulation issues.
                        
  warnings.warn(
/home/maxwellqin/projects/finn/src/finn/transformation/fpgadataflow/create_stitched_ip.py:290: UserWarning: First node is not StreamingFIFO or IODMA.
                You may experience incorrect stitched-IP rtlsim or hardware
                behavior. It is strongly 

In [53]:
from finn.transformation.fpgadataflow.make_pynq_driver import MakePYNQDriver
model = model.transform(MakePYNQDriver("zynq-iodma"))

In [28]:
model.save(build_dir + "/lenet_synth.onnx")

In [29]:
from shutil import copy
from distutils.dir_util import copy_tree
from finn.util.basic import make_build_dir

# create directory for deployment files
deployment_dir = make_build_dir(prefix="pynq_deployment_")
model.set_metadata_prop("pynq_deployment_dir", deployment_dir)

# get and copy necessary files
# .bit and .hwh file
bitfile = model.get_metadata_prop("bitfile")
hwh_file = model.get_metadata_prop("hw_handoff")
deploy_files = [bitfile, hwh_file]

for dfile in deploy_files:
    if dfile is not None:
        copy(dfile, deployment_dir)

# driver.py and python libraries
pynq_driver_dir = model.get_metadata_prop("pynq_driver_dir")
copy_tree(pynq_driver_dir, deployment_dir)

creating /home/maxwellqin/projects/finn/build_output/pynq_deployment_kedrbovv/runtime_weights
creating /home/maxwellqin/projects/finn/build_output/pynq_deployment_kedrbovv/finn
creating /home/maxwellqin/projects/finn/build_output/pynq_deployment_kedrbovv/finn/util
copying /home/maxwellqin/projects/finn/build_output/pynq_driver_qxy7whml/finn/util/data_packing.py -> /home/maxwellqin/projects/finn/build_output/pynq_deployment_kedrbovv/finn/util
copying /home/maxwellqin/projects/finn/build_output/pynq_driver_qxy7whml/finn/util/__init__.py -> /home/maxwellqin/projects/finn/build_output/pynq_deployment_kedrbovv/finn/util
copying /home/maxwellqin/projects/finn/build_output/pynq_driver_qxy7whml/validate.py -> /home/maxwellqin/projects/finn/build_output/pynq_deployment_kedrbovv
copying /home/maxwellqin/projects/finn/build_output/pynq_driver_qxy7whml/driver.py -> /home/maxwellqin/projects/finn/build_output/pynq_deployment_kedrbovv
copying /home/maxwellqin/projects/finn/build_output/pynq_driver_q

['/home/maxwellqin/projects/finn/build_output/pynq_deployment_kedrbovv/finn/util/data_packing.py',
 '/home/maxwellqin/projects/finn/build_output/pynq_deployment_kedrbovv/finn/util/__init__.py',
 '/home/maxwellqin/projects/finn/build_output/pynq_deployment_kedrbovv/validate.py',
 '/home/maxwellqin/projects/finn/build_output/pynq_deployment_kedrbovv/driver.py',
 '/home/maxwellqin/projects/finn/build_output/pynq_deployment_kedrbovv/driver_base.py',
 '/home/maxwellqin/projects/finn/build_output/pynq_deployment_kedrbovv/qonnx/util/__init__.py',
 '/home/maxwellqin/projects/finn/build_output/pynq_deployment_kedrbovv/qonnx/util/basic.py',
 '/home/maxwellqin/projects/finn/build_output/pynq_deployment_kedrbovv/qonnx/core/datatype.py',
 '/home/maxwellqin/projects/finn/build_output/pynq_deployment_kedrbovv/qonnx/core/__init__.py']